# 層のクラスによる実装
本セクションでは、ニューラルネットワークの各層をPythonのクラスとして実装します。  

## 出力層 -回帰-
以下は、回帰の場合の出力層を表すクラスです。   

In [ ]:
# -- 出力層 --
class OutputLayer:
    def __init__(self, n_upper, n):  # 初期設定
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）
    
    def forward(self, x):  # 順伝播
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = u  # 恒等関数
    
    def backward(self, t):  # 逆伝播
        delta = self.y - t
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 

    def update(self, eta):  # 重みとバイアスの更新
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

`OutputLayer`クラスには4つのメソッドが定義されています。  

コンストラクタ（`__init__`メソッド）は初期設定を行います。  
`foward`メソッドは順伝播のメソッドです。  
`backward`メソッドは逆伝播用のメソッドです。  
`update`メソッドは重みとバイアスの更新用のメソッドです。  
 
前のセクションでは関数として層を実装しましたが、クラスにすることで層がより機能的になっています。 

## 出力層 -分類-
分類問題において、出力層は回帰とほぼ同じクラスで実装します。  
唯一の違いは、出力層の活性化関数にソフトマックス関数を使用する点です。  

In [ ]:
class OutputLayer:
    def __init__(self, n_upper, n):  # 初期設定
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）
        
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = np.exp(u)/np.sum(np.exp(u), axis=1, keepdims=True)  # ソフトマックス関数
        
    def backward(self, t):  # 逆伝播
        delta = self.y - t
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 

    def update(self, eta):  # 重みとバイアスの更新
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

sum関数の引数に`keepdims=True`を設定することで、元の配列の次元が保たれます。  
これにより、以下の箇所の計算結果は(バッチサイズ x 1)の行列になります。

```Python
np.sum(np.exp(u), axis=1, keepdims=True) 
```

この計算結果と、`np.exp(u)`の計算結果は行の数がともにバッチサイズで一致しているので、Numpyのブロードキャスト機能を適用し、割り算を行うことができます。  
これにより、バッチに対応したソフトマックス関数になります。  

## 中間層
以下は、回帰、分類共通の中間層を表すクラスです。 

In [ ]:
# -- 中間層 --
class MiddleLayer:
    def __init__(self, n_upper, n):  # 初期設定
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）

    def forward(self, x):  # 順伝播
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = 1/(1+np.exp(-u))  # シグモイド関数
    
    def backward(self, grad_y):  # 逆伝播
        delta = grad_y * (1-self.y)*self.y  # シグモイド関数の微分
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 
        
    def update(self, eta):  # 重みとバイアスの更新
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

出力層との違いは、活性化関数がシグモイド関数である点と、```backward```メソッドで```delta```を求めるのに出力層と異なる式を使用している点です。  
このクラスを用いれば、例えば以下のように中間層をいくつでもインスンスとして生成することができます。

In [ ]:
middle_layer_1 = MiddleLayer(3, 4)
middle_layer_2 = MiddleLayer(4, 5)
middle_layer_3 = MiddleLayer(5, 6)